In [ ]:
%load_ext rpy2.ipython

from rpy2.robjects.packages import importr
import numpy as np
import pandas as pd
from chromax import Simulator
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from tqdm import tqdm
import os
os.environ["KERAS_BACKEND"] = "jax"
import matplotlib
from jax import device_get


In [ ]:

def generate_genetic_map(n_markers, n_chromosomes):
  df = pd.DataFrame(generate_marker_effects(n_markers=n_markers), columns=['Yield'])
  df['cM'] = np.random.uniform(0, 200, len(df))
  df['CHR.PHYS'] = '1A'
  df = df.sort_values(by='cM')
  df = df[['CHR.PHYS', 'cM', 'Yield']]
  # save df as csv under filename
  return df

def simplify_geneticmap(lst, qtl):
    # Select 'qtl' random indexes
    indexes_positive = random.sample(range(len(lst)), qtl)
    
    # Select 'qtl' random indexes not already selected
    remaining_indexes = set(range(len(lst))) - set(indexes_positive)
    indexes_negative = random.sample(remaining_indexes, qtl)

    # Modify the list
    for i in range(len(lst)):
        if i in indexes_positive:
            lst[i] = random.uniform(0.5, 1.0)  # Assign random float between 0.5 and 1.0
        elif i in indexes_negative:
            lst[i] = random.uniform(-0.5, -1.0)  # Assign random float between -0.5 and -1.0
        else:
            lst[i] = 0  # Assign 0

    return lst

def generate_population(n_pop=100, n_markers=500):
    """
    Generate a numpy array of randoms of length 500 with randomized 0, 1, or 2 at each position.
    It will generate 100 individuals based on n_pop.

    Returns: numpy array of size (n_pop, n_markers)
    """
    shape=(n_pop, n_markers, 2)
    # Define the elements to choose from and their associated probabilities
    elements = [0, 1, 2]
    probabilities = [1/3, 1/3, 1/3]  # equal probabilities for 0, 1, and 2

    # Generate the population
    population = np.random.choice(elements, size=(n_pop, n_markers), p=probabilities)

    return np.random.choice([True, False], size=shape)


def generate_marker_effects(n_markers=500, mu=0, sigma=0.1):
    """
    Generate a numpy array of marker effects with a normal distribution.

    Parameters:
    n_markers (int): Number of markers.
    mu (float): Mean of the distribution.
    sigma (float): Standard deviation of the distribution.

    Returns:
    numpy array of marker effects
    """

    # Generate the marker effects
    marker_effects = np.random.normal(mu, sigma, n_markers)

    return marker_effects

def parse_markerEffects(genetic_map, nChr):
    # Get the length of the genetic map
    length = len(genetic_map)

    # Create a new array for storing the chromosome number for each marker
    chr = [0] * length

    # Calculate the number of markers per chromosome
    markers_per_chr = length // nChr

    # Iterate over the range of the genetic map length
    for i in range(length):
        # Calculate the chromosome number and store it in the chr array
        chr[i] = i // markers_per_chr + 1

    return chr


def reshape_pop(maizeHaplo):
    reshapeHaplo = maizeHaplo.reshape(int((maizeHaplo.shape[0])/2),2,maizeHaplo.shape[1])
    reshapeHaplo = reshapeHaplo.transpose((0,2,1))
    return reshapeHaplo

def return_genetic_map_df(markerEffects, nChr, geneticMap):
    chr = parse_markerEffects(markerEffects, nChr)
    chr = [int(x[0]) for x in chr]
    trait = markerEffects
    pos = geneticMap
    # Assuming chr, trait, pos are your arrays
    df = pd.DataFrame({'CHR.PHYS': chr, 'Yield': trait, 'cM': pos})
    return df


In [ ]:
%%R
x <- seq(0, 2*pi, length.out=50)

In [ ]:
%R -o x

In [ ]:
x

In [ ]:
# %R install.packages("AlphaSimR")

In [ ]:
%%R
library("AlphaSimR")

In [ ]:
%%R
nInd = 10
nChr = 2
segSites = 5

founderGenomes = runMacs(nInd = nInd,
                         nChr = nChr,
                         segSites = segSites,
                         species = "MAIZE")


In [ ]:
%%R
SP = SimParam$new(founderGenomes)
SP$addTraitA(segSites)
# SP$setVarE(h2=.02)
pop = newPop(founderGenomes, simParam=SP)
ans = fastRRBLUP(pop, simParam=SP, useQtl=TRUE, use='gv')
ans@gv[[1]]@addEff
markerEffects = slot(slot(ans, "gv")[[1]], "addEff")
maizeHaplo = pullSegSiteHaplo(pop)
maizeGeno = pullSegSiteGeno(pop)
#cm positions of each marker
genMap = SP$genMap
geneticMap = unlist(genMap)

In [ ]:
%R -o maizeHaplo
%R -o maizeGeno
%R -o markerEffects
%R -o geneticMap
%R -o nInd
%R -o nChr
%R -o segSites

In [ ]:
nMarkers = segSites * nChr

In [ ]:
import numpy as np
from tensorflow.keras.layers import Input, Flatten, Dense, Reshape
from tensorflow.keras import Model
import tensorflow as tf
from tensorflow.keras.optimizers import Adam



#dummy functions to  generate fake data to develop the training pipeline
def pop_gen(b, n, m, d):
    return np.random.randint(2, size=(b, n, m, d))
def reward_gen():
    return np.random.rand
def scores_gen(n):
    return np.random.rand(1, n)

import numpy as np
from scipy import stats
def get_r(values):

    # Create an array of indices, same length as your list
    indices = np.array(range(len(values)))

    # Perform linear regression
    slope, intercept, r_value, p_value, std_err = stats.linregress(indices, values)
    return slope

# genetic_map = return_genetic_map_df(markerEffects, nChr, geneticMap)
# genetic_map['Yield'] = simplify_geneticmap(list(genetic_map['Yield']),4)

# n = int(nInd[0])
# m = int(nMarkers)
# d = 2
# total_parents = n*2

# population_dummy = pop_gen(1, n, m, d)  # Extra dimension for batch size
# scores_dummy =  scores_gen(n) # Extra dimension for batch size

# actor_model = create_actor(n,m,d,total_parents, population_dummy, scores_dummy)
# actor_output = actor_model([population_dummy, scores_dummy])
# critic_model = create_critic(n,m,d, population_dummy, scores_dummy,actor_output)

# simulator = Simulator(genetic_map=genetic_map, h2=.5)
# simulator.load_population('mypop.npy')
# initial_scores = simulator.GEBV(population_dummy[0])

# TOTAL_EPISODES=3
# before = print_memory_usage()
# for episodes in range(TOTAL_EPISODES):
#     inputs = tf.random.normal([1, n, m, d]) # replace with actual inputs
#     scores_dummy = tf.random.normal([1, n]) # replace with actual scores
#     #policy, current_population, simulator
#     new_pop,reward = policy2offspring_reward(actor_output, population_dummy, simulator)
#     train_step(actor_model, critic_model, inputs, scores_dummy, reward)
#     tf.keras.backend.clear_session()
# after = print_memory_usage()

# print(f'cost per episode: {( after - before ) / TOTAL_EPISODES}')

In [353]:
from statistics import mean

def split_and_average(lst, set_size):
    # lst is from get_baseline
    #average list
    lst = [x.mean() for x in lst]
    # Split the list into sublists
    split_lst = [lst[i:i + set_size] for i in range(0, len(lst), set_size)]

    # Calculate the average of the x-th element in each sublist
    averages = []
    for x in range(set_size):
        x_elements = [sublist[x] for sublist in split_lst if len(sublist) > x]
        averages.append(mean(x_elements))

    return averages

def sample_two_offspring_per_cross(dataset):
    """
    Samples two offspring per cross from the dataset.

    Parameters:
    dataset (np.ndarray): The input dataset with shape (n_cross, n_offspring, markers, diploid).

    Returns:
    np.ndarray: A new array with sampled offspring, shape (n_cross, 2, markers, diploid).
    """
    n_cross = dataset.shape[0]
    n_offspring_per_cross = 2
    # Create an array to store the indices of the offspring to sample
    indices = np.random.choice(dataset.shape[1], size=(n_cross, n_offspring_per_cross), replace=False)
    # Use advanced indexing to select two offspring per cross
    sampled_offspring = dataset[np.arange(n_cross)[:, None], indices, :, :]
    return sampled_offspring.reshape(-1, sampled_offspring.shape[2], sampled_offspring.shape[3])




def collapse_first_two_axes(dataset):
    """
    Collapses the first two axes of the dataset.

    Parameters:
    dataset (np.ndarray): The input dataset with shape (n_cross, n_offspring, markers, diploid).

    Returns:
    np.ndarray: A new array with the first two axes collapsed, shape (n_cross * n_offspring, markers, diploid).
    """
    # Calculate the new shape
    new_shape = (dataset.shape[0] * dataset.shape[1],) + dataset.shape[2:]
    # Reshape the dataset
    collapsed_dataset = dataset.reshape(new_shape)
    return collapsed_dataset


def select_parents(policy, total_parents=5):
    """
        input: Policy from actor ( metric value for each individual )
        returns : index of parents to be included in random_crosses for next step of breeding program
    """
    values, indices = tf.math.top_k(policy, total_parents)
    return values,indices


In [354]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd  # Import pandas for rolling average

def normalize_data(data, mean, std_dev):
    return (data - mean) / std_dev

def plot_training_progress(actor_losses, critic_losses, average_scores, iteration, agent):
    # Calculate mean and standard deviation of average_scores for normalization
    average_scores_mean = np.mean(average_scores)
    average_scores_std = np.std(average_scores)

    # Normalize each array independently using Z-score normalization
    actor_losses_normalized = normalize_data(actor_losses, average_scores_mean, average_scores_std)
    critic_losses_normalized = normalize_data(critic_losses, average_scores_mean, average_scores_std)
    average_scores_normalized = normalize_data(average_scores, average_scores_mean, average_scores_std)

    # Calculate the window size as 10% of the length of the average_scores array
    window_size = max(int(len(average_scores) * 0.1), 1)  # Ensure at least one element in the window

    # Convert lists to pandas Series
    actor_losses_series = pd.Series(actor_losses_normalized)
    critic_losses_series = pd.Series(critic_losses_normalized)
    average_scores_series = pd.Series(average_scores_normalized)

    # Calculate rolling averages
    actor_losses_rolling = actor_losses_series.rolling(window=window_size).mean().fillna(method='bfill')
    critic_losses_rolling = critic_losses_series.rolling(window=window_size).mean().fillna(method='bfill')
    average_scores_rolling = average_scores_series.rolling(window=window_size).mean().fillna(method='bfill')

    # Normalize baselines using the mean and std dev of average_scores
    random_baseline_normalized = normalize_data(np.array([agent.random_baseline]), average_scores_mean, average_scores_std)
    truncation_baseline_normalized = normalize_data(np.array([agent.truncation_baseline]), average_scores_mean, average_scores_std)

    clear_output(wait=True)
    plt.figure(figsize=(12, 4))

    # Plot all normalized rolling averages on the same plot
    plt.plot(actor_losses_rolling, label='Actor Loss (rolling avg)')
    plt.plot(critic_losses_rolling, label='Critic Loss (rolling avg)')
    plt.plot(average_scores_rolling, label='Average Score (rolling avg)')

    # Add horizontal lines for normalized baselines
    plt.axhline(y=random_baseline_normalized, color='r', linestyle='--', label='Random Baseline (normalized)')
    plt.axhline(y=truncation_baseline_normalized, color='g', linestyle='--', label='Truncation Baseline (normalized)')

    plt.xlabel('Iteration')
    plt.title('Normalized Rolling Averages of Training Progress Over Time')
    plt.legend()

    # Show the plot
    plt.tight_layout()
    plt.show()


In [355]:
from typing import Dict, Any, List


def add_batchdim(arr):
    return np.expand_dims(arr, axis=0)


In [537]:
# FUNCTIONS FO SIMULATING THE ENVIRONMENT AND ACTIONS

def create_fake_geneticmap(number_markers):
    # 'chr' will always be '1A' for every marker
    chr_array = ['1A'] * number_markers
    
    # 'yield': Create a marker_strength array with 1 float between -0.5 and +0.5 randomly
    yield_array = np.random.poisson(5, size=number_markers)
    
    # 'cM': create an array for number_markers length evenly sampled between 0 and 100
    cM_array = np.linspace(0, 100, num=number_markers)
    
    # Create the DataFrame with the auto-generated data
    df = pd.DataFrame({'CHR.PHYS': chr_array, 'Yield': yield_array, 'cM': cM_array})
    
    return df

def create_fake_population(total_pops, number_individuals, number_markers, ploidy):
    # Generate a random population array with values 0 or 1
    population = np.random.randint(2, size=(total_pops,number_individuals, number_markers, ploidy))
    return population


def calculate_scores(population, marker_strength):
    """
    Calculates the additive score by matrix multipling the population (n,m) with the marker strengths (m,)
    """
    # Perform a dot product between the dosages and marker strength arrays
    dosages = np.sum(population,axis=2)
    scores = np.dot(dosages, marker_strength)
    return scores

def panmixia(selected_parents, total_offspring = selected_parents.shape[0]*2):
    """ Handles the random crossing for us ; heuristic!

    selected_parents ( n , m , d )
    
    """
    n, m, d = selected_parents.shape
    offspring_target = total_offspring
    offspring_list = []

    while len(offspring_list) < offspring_target:
        # Randomly pick two parents without replacement
        parents_indices = np.random.choice(n, size=2, replace=False)
        parent1, parent2 = selected_parents[parents_indices]

        # Simulate random recombination for each marker
        offspring = np.zeros((m, d), dtype=parent1.dtype)
        for i in range(m):
            # Randomly choose one allele from each parent for each marker
            for j in range(d):
                parent_allele = np.random.choice([parent1[i, j], parent2[i, j]])
                offspring[i, j] = parent_allele

        # Add the new offspring to the offspring list
        offspring_list.append(offspring)

    # Convert offspring list to a numpy array with shape (offspring_target, m, d)
    offspring_array = np.array(offspring_list)
    return offspring_array


def scores2parents(scores,K):
    """
    hint: use output from calculate_scores
    """
    # Specify the number of top values you want (K)
    K = 5

    # Get the indices that would sort the array
    sorted_indices = np.argsort(scores)

    # Take the last K indices of the sorted indices array
    top_k_indices = sorted_indices[-K:]

    # Since argsort returns indices in ascending order, reverse to get the top values
    top_k_indices = top_k_indices[::-1]

    print("Indices of top K values:", top_k_indices)
    print("Top K values:", scores[top_k_indices])

    return top_k_indices

In [538]:
config = {
    'compute_baselines': False,
    'initial_population': initial_population,
    'ploidy':ploidy,
    'genetic_map': genetic_map_df,
    'population_size': number_individuals,
    'marker_count': number_markers,
    'heritability': .99,
    'episodes': 10,
    'cycles': 2,
    'learning_rate': .001,
    'replicates': 1,
    'actor_lr':.00005,
    'critic_lr':.0000005, 
}

In [539]:
# Example usage:
number_individuals = 50  # Replace with your actual number of individuals
number_markers = 100     # Replace with your actual number of markers
ploidy = 2              # Replace with your actual ploidy level
batch_size = 5

initial_population = create_fake_population(batch_size, number_individuals, number_markers, ploidy)
genetic_map_df = create_fake_geneticmap(number_markers)
marker_strength = np.array(genetic_map_df['Yield'])

dummy_score = np.random.rand(batch_size, number_individuals)

actor_model = create_actor_model(number_individuals, number_markers, ploidy)
critic_model = create_critic_model(number_individuals, number_markers, ploidy, actor_model)

In [540]:
policy = actor_model([initial_population,dummy_score])
value_estimate = critic_model([initial_population, dummy_score])

In [541]:
#split the population in half based on the policy metric
top_policy_parents = select_parents(policy[batch_id,:].numpy(), total_parents=int(initial_population.shape[1] /2))[1].numpy()

bottom_policy_parents = np.arange(initial_population.shape[1])
bottom_policy_parents = np.setdiff1d(bottom_policy_parents, top_policy_parents)

top_policy_parents = initial_population[batch_id][top_policy_parents]
bottom_policy_parents = initial_population[batch_id][bottom_policy_parents]


In [542]:
true_scores = calculate_scores(initial_population[batch_id], marker_strength)

In [543]:
calculate_scores(panmixia(bottom_policy_parents, 500), marker_strength).mean()

476.58

In [544]:
new_pop = panmixia(top_policy_parents, 50)
new_scores = calculate_scores(new_pop, marker_strength)
print(new_scores.mean())
new_parents = scores2parents(new_scores,25)

478.2
Indices of top K values: [41 17 24 36 39]
Top K values: [578 569 557 551 534]


In [545]:
new_pop = panmixia(new_pop[new_parents], 50)
new_scores = calculate_scores(new_pop, marker_strength)
print(new_scores.mean())
new_parents = scores2parents(new_scores,2)

562.32
Indices of top K values: [33 24 41  5 38]
Top K values: [654 641 621 607 606]


In [546]:
print(
    f'''
    Batch Size : {batch_size}
      
    Population Size : {number_individuals}
    Marker Count : {number_markers}

    Population Input Shape : {initial_population.shape}
    Scores Input Shape : {dummy_score.shape}

    Policy Output Shape : {policy.shape}
    Values Output Shape : {value_estimate.shape}
  
    '''
      )


    Batch Size : 5
      
    Population Size : 50
    Marker Count : 100

    Population Input Shape : (5, 50, 100, 2)
    Scores Input Shape : (5, 50)

    Policy Output Shape : (5, 50)
    Values Output Shape : (5, 1)
  
    


In [586]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate, Lambda

def create_actor_model(num_individuals, num_markers, ploidy):
    # Define the input layers
    population_input = Input(shape=(num_individuals, num_markers, ploidy), name='population_input')
    scores_input = Input(shape=(num_individuals,), name='scores_input')

    # Flatten the population input to process it with Dense layers
    flat_population = Flatten()(population_input)

    # Combine the flattened population and scores inputs
    combined_input = Concatenate()([flat_population, scores_input])

    # Define the hidden layers
    hidden1 = Dense(128, activation='relu')(combined_input)
    hidden2 = Dense(64, activation='relu')(hidden1)

    # Output layer with one scalar value per individual
    policy_output = Dense(num_individuals, activation='sigmoid', name='policy_output')(hidden2)

    # Create the actor model
    actor_model = Model(inputs=[population_input, scores_input], outputs=policy_output)

    return actor_model

def create_critic_model(num_individuals, num_markers, ploidy, actor_model):
    # Define the input layers
    population_input = Input(shape=(num_individuals, num_markers, ploidy), name='population_input')
    scores_input = Input(shape=(num_individuals,), name='scores_input')

    # Call the actor model to get the policy output for the given state
    policy_output = actor_model([population_input, scores_input])

    # Use Lambda layer to apply tf.stop_gradient to prevent backpropagation
    policy_output_no_gradient = Lambda(lambda x: tf.stop_gradient(x))(policy_output)

    # Flatten the population input to process it with Dense layers
    flat_population = Flatten()(population_input)

    # Combine the flattened population, scores, and policy outputs without gradients
    combined_input = Concatenate()([flat_population, scores_input, Flatten()(policy_output_no_gradient)])

    # Define the hidden layers
    hidden1 = Dense(128, activation='relu')(combined_input)
    hidden2 = Dense(64, activation='relu')(hidden1)

    # Output layer with a single value estimate for the state
    value_output = Dense(1, activation='linear', name='value_output')(hidden2)

    # Create the critic model
    critic_model = Model(inputs=[population_input, scores_input], outputs=value_output)

    return critic_model




class BreederAgent:
    """
    A PPO agent that learns to select parents for breeding programs using Actor-Critic method.
    """
    
    def __init__(self, config: Dict[str, Any]):
        """
        Initializes the breeding program with a configuration dictionary.
        
        Parameters:
        config (dict): Configuration dictionary containing all necessary parameters.
        """

        # Initialize the basic attributes from the config
        self.compute_baselines = config.get('compute_baselines', False)
        self.population_size = config['population_size']
        self.marker_count = config['marker_count']
        self.initial_population = config['initial_population']
        self.genetic_map = config['genetic_map']
        self.marker_strength = np.array(self.genetic_map['Yield'])

        self.ploidy = config['ploidy']


        # Initialize the Actor and Critic models
        self.actor = create_actor_model(num_individuals=self.population_size, num_markers=self.marker_count, ploidy=self.ploidy)
        self.critic = create_critic_model(num_individuals=self.population_size, num_markers=self.marker_count, ploidy=self.ploidy, actor_model=self.actor)


        #truncation and random baselines
        # self.truncation_baseline = [calculate_scores(x, self.marker_strength) for x in self.initial_population]
        # truncation_baseline = self.breeding_simulator.select(self.initial_population, k = int(self.initial_population.shape[0]/2))
        # self.truncation_baseline = self.breeding_simulator.GEBV(collapse_first_two_axes(self.breeding_simulator.random_crosses(truncation_baseline, n_crosses = truncation_baseline.shape[0], n_offspring=50))).mean()[0]

        # self.random_baseline = self.breeding_simulator.GEBV(collapse_first_two_axes(self.breeding_simulator.random_crosses(self.initial_population, n_crosses = self.initial_population.shape[0], n_offspring=50))).mean()[0]


    def reset_environment(self):
        self.current_population = self.initial_population
        self.current_scores = self.initial_scores



In [609]:
# Example usage:
number_individuals = 50  # Replace with your actual number of individuals
number_markers = 100     # Replace with your actual number of markers
ploidy = 2              # Replace with your actual ploidy level
batch_size = 5

initial_population = create_fake_population(batch_size, number_individuals, number_markers, ploidy)
genetic_map_df = create_fake_geneticmap(number_markers)
marker_strength = np.array(genetic_map_df['Yield'])


config = {
    'compute_baselines': False,
    'initial_population': initial_population,
    'genetic_map': genetic_map_df,
    'population_size': number_individuals,
    'marker_count': number_markers,
    'heritability': .99,
    'episodes': 10,
    'cycles': 2,
    'learning_rate': .001,
    'replicates': 1,
    'ploidy':2,
}

agent = BreederAgent(config)


In [610]:
scored_populations = [calculate_scores(x, agent.marker_strength) for x in agent.initial_population]
selected_parents = [select_parents(x)[1].numpy() for x in scored_populations]
subset_parents = [s[p] for s,p in zip(initial_population,selected_parents)]

In [612]:
subset_parents[0].shape

(5, 100, 2)

In [613]:
panmixia(subset_parents[0], total_offspring=500)

array([[[1, 1],
        [1, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 1],
        [0, 0]],

       [[1, 1],
        [1, 0],
        [1, 0],
        ...,
        [0, 1],
        [0, 1],
        [0, 1]],

       [[1, 1],
        [0, 1],
        [1, 0],
        ...,
        [0, 0],
        [1, 1],
        [1, 0]],

       ...,

       [[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 1],
        [1, 0]],

       [[1, 1],
        [1, 0],
        [1, 0],
        ...,
        [0, 1],
        [0, 1],
        [1, 0]],

       [[0, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 1],
        [1, 1],
        [1, 0]]])

In [589]:
agent.marker_count

100

In [590]:

agent.marker_strength.shape

(100,)